In [1]:
# pip install pydantic

In [2]:
# pip install boto3

In [3]:
# pip install gspread

In [4]:
# pip install openai

In [5]:
import sys
import os


repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_root)
print("Added to PYTHONPATH:", repo_root)

Added to PYTHONPATH: /home/jovyan/work


In [6]:
import json
from typing import Any, Dict

import boto3
import yaml

from core.task_processor import TaskProcessor
from core.process_registry import ProcessRegistry
from core.task_publisher import TaskPublisher

from core.processes.client_bank_statement import ClientBankStatementProcess

In [7]:
ENV = "dev"
tenant_id = "alfa"

with open(f"../config/tenants/{tenant_id}/{ENV}.yaml", "r") as f:
    tenant_config: Dict[str, Any] = yaml.safe_load(f)

dynamodb_config = tenant_config.get("dynamodb", {})
dynamodb = boto3.resource(
    "dynamodb",
    region_name=dynamodb_config.get("region"),
)
messages_table = dynamodb.Table(dynamodb_config.get("messages_table"))
processes_table = dynamodb.Table(dynamodb_config.get("processes_table"))
contacts_table = dynamodb.Table(dynamodb_config.get("contacts_table"))

sqs_config = tenant_config.get("sqs", {})
sqs = boto3.client("sqs", region_name=sqs_config.get("region"))
queue_url = sqs_config.get("tasks_url")

In [8]:
task_publisher = TaskPublisher(sqs, queue_url)

processor = TaskProcessor(
    tenant_config=tenant_config,
    messages_table=messages_table,
    processes_table=processes_table,
    contacts_table=contacts_table,
    task_publisher=task_publisher,
)

In [9]:
body = {
    "task_type": "READ_INCOMING_WHATSAPP_MESSAGE", 
    "agent_type": "ACCOUNTING_ASSISTANT", 
    "process_type": "READ_INCOMING_WHATSAPP_MESSAGE", 
    "context_key": "whatsapp:525571969848", 
    "payload": {
        "text": "Tambi\u00e9n me gusta cantar"
    }, 
    "debounce_policy": {"type": "messages_idle", "min_idle_seconds": 60}, 
    "timestamp_iso": "2025-12-13T04:30:05", 
    "timestamp_epoch": 1765600205
}
task = json.dumps(body)

In [10]:
processed, remaining = processor.process(task)

{'identity': 'whatsapp:525571969848', 'contact_id': 'c_55e6d9b66c8b', 'profile': {'updated_at': '2025-12-13T19:15:40Z', 'contact_id': 'c_55e6d9b66c8b', 'created_at': '2025-12-13T19:15:40Z', 'dob': '1993-02-07', 'locale': 'es-MX', 'pk': 'CONTACT#c_55e6d9b66c8b', 'display_name': 'Juan Ignacio Fernández Lrriera', 'sk': 'PROFILE'}, 'summary': {'overview': 'Cliente activo. Comunicación clara y directa.', 'preferences': ['Mensajes cortos', 'WhatsApp como canal principal'], 'special_notes': ['En diciembre responde más lento'], 'constraints': ['Si amount > 0, pedir confirmación por mail']}}
[PrintService to:525571969848] Hola Nacho, gracias por compartir conmigo que sos morocho, naciste en Argentina, tenés 32 años, sos programador, te gusta mucho el helado, cantar, tu color favorito es el azul y tus lugares preferidos son Uruguay, Argentina y México. Espero que estés disfrutando tu sábado. Si necesitás algo, estoy aquí para ayudarte. ¡Que tengas un excelente día!
